# CKAN API GUIDE 

In the following we will demonstrate the usage of the CKAN-API for the Datenportal of the metroplitan area rhine-neckar.
This notebook only contains examples of what we think will be the most common use cases. If you want to know more please visit the [CKAN-API guide]("https://docs.ckan.org/en/2.9/api/"). If you can't find what you are looking for there please contact us directly.
#### Please note:  

Because of the naming of our ressources you can't query for them directly (all of them are named like their format e.g. GeoJSON or GPKG). You need to search the corresponding package and extract the resource-url form there. The process of doing so is demonstratet in a few of the following examples.

In [1]:
import requests
import geopandas as gpd

### Get all package ids

In [2]:
url = "https://daten.digitale-mrn.de/api/3/action/package_list"

res = requests.get(url).json()

# show the first 20
print(res['result'][:20])

['abfallkorb', 'abwasseranlage', 'alla_hopp', 'allgemeinaerzte', 'altenquotient', 'anteil_auslaender_gemeinde', 'anteil_auslaender_kreis', 'apotheke', 'arbeitsentgeld', 'arbeitslosenquote', 'arbeitslosenquote_zivile_erwerbspersonen', 'arbeitsvolumen_jahresdurchschnittliche_aenderung', 'attraktion', 'aussenwanderungssaldo', 'autobahnaehnlich', 'autobahnen', 'autovermietung', 'backerei', 'battlefield', 'bauhaus']


### Get a dataset by id

In [3]:
url = "https://daten.digitale-mrn.de/api/3/action/package_show"

# insert package id here, in this example we will request the package "autbahnen"
params = {
    'id':'autobahnen'
}

res = requests.get(url, params).json()

data = res['result']

########### create a geodataframe from the ressource ###########

# ectract geoserver url from the resource of the dataset
# Note: resources is an array and there can be multiple resources (GeoJSON, GPKG, WMS, CSV) for the same package,
# we want the goejson one.
for format_ in data['resources']:
    # insert wanted format here
    if(format_['format'] == 'GeoJSON'):
        #extract geoserver url from ressource package
        url_dataset = format_['url']

        
        # request data from geoserver
        res_dataset = requests.get(url_dataset)
        
        # extract features form geojson
        dataset_features = res_dataset.json()['features']

        # create new gdf from features
        gdf = gpd.GeoDataFrame.from_features(dataset_features)

gdf.head(2)

,geometry,Name,highway,int_ref,lanes,lit,maxspeed,oneway,overtaking.hgv.conditional,overtaking.trailer.conditional,...,sidewalk,maxspeed.wet,destination.symbol.lanes,source.hazard,incline,service,placement.forward,construction.maxspeed.conditional,toll.N3,transit.lanes
0,"LINESTRING (8.62752 49.54889, 8.62762 49.54792...",n/a,motorway,E 35,2,no,none,yes,no @ (06:00-21:00),no @ (06:00-21:00),...,None,None,None,None,None,None,None,None,None,None
1,"LINESTRING (8.63166 49.59335, 8.63143 49.59447...",n/a,motorway,E 35,3,no,none,yes,no @ (06:00-21:00),no @ (06:00-21:00),...,None,None,None,None,None,None,None,None,None,None


### Search for packages matching a query

In [4]:
url = "https://daten.digitale-mrn.de/api/3/action/package_search"


# search for packages with the substring LuftDatenInfo in title
params = {
    'q':'title:LuftDatenInfo'
    
}

# search for specific title. Note: space characters must be replaced by '-'
params_2 = {
    'q':'title:LuftDatenInfo-Feinstaubemissionen-der-Partikelgröße-PM2.5'
}

# you can also query for ids. Note: in the query syntax id is called name
params_3 = {
    'q':'name:autobahnen'
    
}

res = requests.get(url, params_3).json()

print(res['result'])

{'count': 1, 'sort': 'score desc, metadata_modified desc', 'facets': {}, 'results': [{'license_title': 'Open Data Commons Open Database License (ODbL)', 'maintainer': '', 'relationships_as_object': [], 'private': False, 'maintainer_email': '', 'num_tags': 2, 'id': 'd11748f0-1e24-44d6-826c-5d3a9b374630', 'metadata_created': '2020-03-03T09:31:32.306214', 'metadata_modified': '2020-06-03T11:49:18.982506', 'author': '', 'author_email': '', 'state': 'active', 'version': '', 'creator_user_id': '70930133-5270-4d89-8270-558800f0f75a', 'type': 'dataset', 'resources': [{'mimetype': None, 'cache_url': None, 'hash': '', 'description': '', 'name': 'GeoJSON', 'format': 'GeoJSON', 'url': 'https://geoserver.digitale-mrn.de/geoserver/metropolatlas/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=metropolatlas%3A2019-10_OSM_MRN_highway_motorway_4326&outputFormat=application%2Fjson', 'cache_last_updated': None, 'package_id': 'd11748f0-1e24-44d6-826c-5d3a9b374630', 'created': '2020-03-09T13:53:01

### Get a list of all tags

In [5]:
url = "https://daten.digitale-mrn.de/api/3/action/tag_list"

res = requests.get(url).json()

print(res['result'])

['Auto', 'Bevölkerung', 'Bildung', 'Daseinsvorsorge', 'Demograpie', 'Digitalisierung', 'Ehrenamt', 'Elektromobilität', 'Engagement', 'Events', 'Fahrrad', 'Familie', 'Gastronomie', 'GeoJSON', 'Gesundheit', 'Grenzen', 'Infrastruktur', 'Kriminalität', 'Kultur', 'Ladesäulen', 'Lebensqualität', 'Mobilität', 'Motorisierter Individualverkehr', 'MRN', 'Natur', 'ÖPNV', 'Parken', 'Planen u. Bauen', 'Sharing Economy', 'Sport', 'Tourismus', 'Umwelt', 'Verkehr', 'Verkehrsinfrastruktur', 'Verkehrsmodell', 'Verwaltungseinheiten', 'Wirtschaft', 'Zulassungszahlen']


### Download all ressources for a specific tag

In [7]:
url = "https://daten.digitale-mrn.de/api/3/action/package_search"

# filter for datasets with tag "Verkehrsmodell"
# rows is the parameter to extend the request limit, default = 20, max = 1000
params = {'fq':'tags:Verkehrsmodell',
         'rows': '1000'}

res = requests.get(url, params).json()

data = res['result']['results']


########### extract resource url, reuqest geojson data and save to disc ###########
for dataset in data:

    # there can be multiple resources for one dataset, we want the geojson one
    for format_ in dataset['resources']:
        if(format_['format'] == 'GeoJSON'):
            
            # ectract geoserver url from the resource of the dataset
            url_dataset = format_['url']
            # request data from geoserver
            res_dataset = requests.get(url_dataset)
            
            # extract features form geojson
            try:
                dataset_features = res_dataset.json()['features']
            except:
                print(dataset)
            # create new gdf from features
            gdf = gpd.GeoDataFrame.from_features(dataset_features)
            
            # create file name from geoserver layer name
            file_name = url_dataset.split('metropolatlas%3A')[1]
            file_name = file_name.split('&')[0]
            
            # save file, can be changed to shapefile, geopackage...
            gdf.to_file(file_name+'.geojson', driver='GeoJSON')

{'license_title': 'Open Data Commons Open Database License (ODbL)', 'maintainer': '', 'relationships_as_object': [], 'private': False, 'maintainer_email': '', 'num_tags': 3, 'id': '1fb951f9-dd0e-4bfc-b87b-06f8c3d8a49d', 'metadata_created': '2020-04-15T10:21:06.601922', 'metadata_modified': '2020-10-21T10:04:48.100700', 'author': '', 'author_email': '', 'state': 'active', 'version': '', 'creator_user_id': '70930133-5270-4d89-8270-558800f0f75a', 'type': 'dataset', 'resources': [{'mimetype': None, 'cache_url': None, 'hash': '', 'description': '', 'name': 'GeoJSON', 'format': 'GeoJSON', 'url': 'https://geoserver.digitale-mrn.de/geoserver/metropolatlas/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=metropolatlas%3A2020_04_OSM_Skript_Education_MRN&maxFeatures=50&outputFormat=application%2Fjson', 'cache_last_updated': None, 'package_id': '1fb951f9-dd0e-4bfc-b87b-06f8c3d8a49d', 'created': '2020-04-15T10:21:37.140951', 'state': 'active', 'mimetype_inner': None, 'last_modified': None,